In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import gc
import xgboost as xgb
import math
from joblib import Parallel, delayed

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df: 
    @param threshold: 
    @return: 
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [3]:
# train_df = pd.read_hdf('../input/train.h5')
test_df = pd.read_hdf('../input/test.h5')
sub = pd.DataFrame(test_df['time'])

In [4]:
# train_df = train_df[train_df['temperature'].notnull()]
# train_df = train_df.fillna(method='bfill')
test_df = test_df.fillna(method='bfill')
gc.collect()

31

In [5]:
# train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
#                     'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [6]:
x_train = pd.read_hdf('../input/train_features.h5')
x_test = pd.read_hdf('../input/test_features.h5')
y_train = pd.read_hdf('../input/y_train.h5')

In [7]:
col_corr = correlation(x_train, 0.98)
# print(col_corr)

In [8]:
x_train.drop(list(col_corr), axis=1, inplace=True)
x_test.drop(list(col_corr), axis=1, inplace=True)

In [9]:
x_train.to_hdf('../input/train_features_corr.h5', 'df')
x_test.to_hdf('../input/test_features_corr.h5', 'df')

In [10]:
nums = int(x_train.shape[0] * 0.8)

trn_x, trn_y, val_x, val_y = x_train[:nums], y_train[:nums], x_train[nums:], y_train[nums:]

train_matrix = xgb.DMatrix(trn_x, label=trn_y, missing=np.nan)
valid_matrix = xgb.DMatrix(val_x, label=val_y, missing=np.nan)
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

params = {
    'booster': 'gbtree',
    'eval_metric': 'mae',
    'min_child_weight': 5,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.05,
    'seed': 2020,
    'nthread': 36,
    'silent': 1
}

watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

model_eval = xgb.train(params,
                       train_matrix,
                       num_boost_round=50000,
                       evals=watchlist,
                       verbose_eval=500,
                       early_stopping_rounds=1000)
val_pred = model_eval.predict(valid_matrix, ntree_limit=model_eval.best_ntree_limit).reshape(-1, 1)

[0]	train-mae:0.69791	eval-mae:0.30856
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 1000 rounds.
[500]	train-mae:0.02331	eval-mae:0.15512
[1000]	train-mae:0.01210	eval-mae:0.15504
[1500]	train-mae:0.00745	eval-mae:0.15482
[2000]	train-mae:0.00497	eval-mae:0.15438
[2500]	train-mae:0.00352	eval-mae:0.15427
[3000]	train-mae:0.00255	eval-mae:0.15419
[3500]	train-mae:0.00189	eval-mae:0.15421
[4000]	train-mae:0.00144	eval-mae:0.15412
[4500]	train-mae:0.00110	eval-mae:0.15415
[5000]	train-mae:0.00084	eval-mae:0.15416
Stopping. Best iteration:
[4017]	train-mae:0.00142	eval-mae:0.15412



In [11]:
mse = mean_squared_error(val_y, val_pred)
print("mse_score:", mse)
print("mse_score:", str(mse)[2:7])

mse_score: 0.03863401452251203
mse_score: 03863


In [12]:
feat_imp_dict = model_eval.get_score(importance_type='gain')

In [13]:
feat_imp = pd.Series(feat_imp_dict).sort_values(ascending=False).reset_index().rename(columns={'index': 'feature',
                                                                                               0: 'importance'})
feat_imp['normalized_importance'] = feat_imp['importance'] / feat_imp['importance'].sum()
feat_imp['cumulative_importance'] = np.cumsum(feat_imp['normalized_importance'])

# feat_imp.to_csv('../input/feat_imp.csv', index=False)

In [14]:
record_low_importance = feat_imp[feat_imp['cumulative_importance'] > 0.90]
to_drop = list(record_low_importance['feature'])
print(to_drop)

['MDH_outdoorAtmo_mean_MDH_outdoorHum_diff_max_subtract', 'MD_outdoorHum_sum_MDH_outdoorTemp_sum_multyply', 'MD_outdoorTemp_min_MD_outdoorAtmo_diff_sum_subtract', 'MD_outdoorHum_max', 'MD_outdoorAtmo_diff_sum_MDH_indoorHum_diff_max_subtract', 'outdoorTemp_MDH_outdoorTemp_medi_multyply', 'indoorHum_MDH_outdoorHum_medi_multyply', 'outdoorHum_MDH_indoorHum_diff_sum_ratio', 'MDH_indoorHum_diff_sum_MD_outdoorHum_diff_min_multyply', 'MD_indoorHum_diff_sum_MDH_outdoorHum_min_multyply', 'outdoorTemp_200_bin_indoorHum_std', 'MDH_outdoorTemp_min_MDH_indoorAtmo_mean_subtract', 'MD_outdoorAtmo_diff_sum_MDH_indoorHum_diff_sum_multyply', 'outdoorTemp_100_bin_indoorAtmo_std', 'MDH_indoorAtmo_min_MD_indoorAtmo_diff_sum_multyply', 'MDH_outdoorHum_diff_max_MDH_indoorHum_diff_sum_subtract', 'MD_outdoorAtmo_mean_MDH_indoorAtmo_medi_multyply', 'outdoorHum_MDH_outdoorAtmo_diff_sum_subtract', 'MDH_outdoorHum_diff_min_MD_outdoorAtmo_diff_sum_subtract', 'outdoorHum_100_bin_outdoorAtmo_mean', 'outdoorHum_2D_rol

In [15]:
x_train.drop(to_drop, axis=1, inplace=True)
x_test.drop(to_drop, axis=1, inplace=True)
gc.collect()

40

In [16]:
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

model = xgb.train(params,
                  train_all_matrix,
                  num_boost_round=model_eval.best_ntree_limit + 20)

test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)

In [17]:
sub['temperature'] = test_pred[:, 0] + test_df['outdoorTemp'].values
sub.to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), str(mse)[2:7]), index=False)